Here, I would like to create a notebook to explain how andrej karpathy implement the "minimal character rnn" model 


Andrej Karpathy created a "minimal character rnn" model using python, though he puts some comments on this script, I would like to explain it further to make it more clear.

In [1]:
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 3641 characters, 71 unique.


In the preceding cell, we import numpy and read in our data from the input.txt. Here,I used a article from ftchinese called "The Life of a Song: ‘Sunshine of Your Love" as our input file

then, we create the list "chars" to include all unique words, and as we can see, there are 3641 characters in total, but only 71 unique words.

last, we created two dictionaries, first one is to map each unique word to integer, and second one is to map each unique integer to words

In [2]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias


In the above cells, we are going to initialize some hyperparameters as well as model parameters



In [6]:

def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in xrange(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(xrange(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

here, what we do is to implement the loss function

In [7]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in xrange(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes


In [8]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print '----\n %s \n----' % (txt, )

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 E�GKDzTw(kmLC7N-kDtm5b(wj)6,Rt�vS2Y�
LsehYfdIj1N5I
wmHYtvo1NqAl�F�tY�tkTRNaMHg(Ea
MtG;�Hcwn7�cxtmw862mS
tiLLu�
ewomPS�pdzPuW,iY��FoWe�GG-vdd
sjBsR2Bk2T:,2o92G 0wnc7euDh6KLa(dJ;KvF4;mlk,5fz�;o,u�mtu7Kc 
----
iter 0, loss: 106.566998
----
   e � t bHeklfrB ur�it r�coon�rwoaMtks �:� rcr�vh pf t�  r  tswt �pm�igs �r ltor  ,anri� a nar   sc oi n H  fard gp  r  i  osfaeH�t nag sr�b � ca �a  eb re r f faaasf r h rar �wan H s cd gb tt�rrce�rc 
----
iter 100, loss: 106.982107
----
  w n ntentniSebtn k�ki 6t:iie �sn�tiehpeogte�e igamnr iwt pnt:htbhthewoeyan s e hevrm eggehe eng  heanginercpniw:nM�g6n sttmtor.nvnefgisve herertt,esilu igm:nnlehwhe ,tpshb gh�gohco itnton eninghnelto 
----
iter 200, loss: 105.432642
----
  ��fpnnu H
s tho �H�e�dpl yeelshpsy  JwtoobHesneHo Hr e ��w- w anlwJTa cF, yHcHeye xrpa erc e Hsp�ol6tSar. un�thl �auhe ex�wdoH5 �Je J  1s Hed�w e � poJCeJd rypnyn ,aAesrilere oy aywpy cLep.�. �u lreL 
----
iter 300, loss: 103.247270
----
 nnreanflako te,emof t,kmo�moeifari k

----
 wlyar corind fnay briniis oul. luthes agded-ribliddrols he ricz Bliterered on th sh gleatext.ring-erritad ar Yoon Jith wifpl, thet thohragg the blar, torer inat the boodsesthitithamd pext vliberays ge 
----
iter 3500, loss: 57.472870
----
 ��Sur Royl st riwo elyrirappad ghit.
Mfes LeJe -hor len ’ssil, velilecerictaf pithe, shose Auk foherer wher, sifattemdrir wovsir,. The” oogte Al” Araloe the soye tiof Nowa, rithe the plamin” s 
----
iter 3600, loss: 56.881716
----
  er led in Cr third o. Ytre “Ind ursmnge;d and, Crerns to tin he Wham Bls, dln” Br — ers guth-tusg wors in thad dlig uis mion theg the herde bouse, sis of yiring in Thal sost wis ponJpred in Bla 
----
iter 3700, loss: 56.356854
----
 berof the surithe Hon the riva on stord cate” Line ouc Lvo toring thit pricy Hrocore they tree fh” nuved ouc, U�s e, Wil“Suterickowd the livean ang on Geate st�zSr, “Sume of ay Londrok“Iorm 
----
iter 3800, loss: 55.910511
----
 feibarr, inher a ’s al the sers nsod Gind an the .��s er bt

----
  sroming wast rame ae tt Tha somat s ig th a  ong beanixt Brid he thed hick a wa. Batere ar a on the peny shott moung rum awr , andHe A Ha whoor.
Ris ce wad buneme an” a ste comway th keat rind ghe  
----
iter 7000, loss: 46.019353
----
 ar a spindrud aplalibeag U5 cean” pund-timhen a se tiastert a Cy a so Gine stastapengeri at the geanemas toucering in the to Creak that, ar teicitaste the ensb. Clis the shastin ha lue sing piom siv 
----
iter 7100, loss: 45.734524
----
 eat the bmeatsr fosgurubiver Royenculle tot bandsx ftoin he andrinl Lat bulu uleysrtamdsuge — al, the sons on ass—  umain tre, fivenpe sihe ns am te toc“Sunseride sut toter” llcy a fist asd, s 
----
iter 7200, loss: 45.533757
----
 e taleror ther — Creming coce Jast hred hesth wine rist Jive ho potir ensendeiplestick onone Broyels sencer on prim whin he gheas tingew Hame vep itz of lstout lisemins Lnin Dos thosid gensecsly ir  
----
iter 7300, loss: 45.295872
----
 drome, fick-buid Amlithe Amlic tering bors on an

----
 s deceuthe and as the song plotby noullylillly a shistrudh wo whting a Cliriche — enstinatrt a to eroing the rixt woon vetsith writeging in the pisits on Brumed of toule”. Foiricowe Broms of Youne 
----
iter 10500, loss: 40.041052
----
 rd of the initblenthe fatk they frot. Whot-tle he Jirias it. Fetsfhen thed thave hizzsta Brkyeat thwr is inandrake, to thas son, ke. Brucanr, grucerd, US that Sica, Bubce’s Lanpliald a Apcegald-mns  
----
iter 10600, loss: 39.908210
----
  fistong on Hew. Anso droyl, hauletec of Clienst trimmonesrodicas.” aed castsriasto oory belaln ghoed of Clawhatias vinger sowsl,  fige of Yot aFstiam a to ludins mame to ” frrital of the anderthe 
----
iter 10700, loss: 39.734438
----
 rownyepindrix lpling ig ta treWillysicked vine;rew ffows tribe if Cre bco ston epte sedeshist ritodix Fot byoly ngiguown It: as oo if -frye the read tiat Bion  ist-ridisisthe”; hisediwis toing if Yo 
----
iter 10800, loss: 39.652151
----
  meysias iast ome coully lineming of Youle

----
 o ant of Soneris kyins. Fourning was wor Amby Girut kray on ther ancole so’d ithar of “Itrove. comd of Noriso cour Gith Neg of wast. Indw.
Eld Bronthe toringerm, om theing herte the sor trimm. Ies 
----
iter 14000, loss: 36.141249
----
 ric th protred nguteoor to “Sunsurnge Crapled ote hes rome’s wa bed cotoughe singil, trown lroma sowe song on the shith wher ’d a beest”. Madm way son to Brooto Alom songn felplithe siwnfmr C� 
----
iter 14100, loss: 36.024426
----
 ly ias “Sunshine, to Yhay to als in and rione atammang veaten;le pfalmes — hals, Creame of the soms sume and diot ereit thit neast rave song gus of Youn Brores fmam CliMarin’s lave, A ames —.  
----
iter 14200, loss: 35.840038
----
  bere sooteas at, Brown;nin tr berdeuthat rruke”.6F the haplidut bounspind the styles thit the ringe wo work a Hapllat versoinge of tive so a digza liven;frak writat it the ble sereataye they tammar 
----
iter 14300, loss: 35.741209
----
 on persuon’s on thir A Hersion Negtiget byixp boup, 

...

----
 gus but beage the first to Br when — the by; ts instily besther Im the US and rine. Americnte, addibupeng was they thise, as the riff plappensendwe’s llive showam the song a Clmame of poully-ncord 
----
iter 280000, loss: 2.156631
----
 dindurining to weron gint tied ofer. Bbocgesd bane of Fithe rourd insteall-ting list haste, “We’t enia povon — hase promeed of fathid out the riff the chorus.
But the song was still not finis fo 
----
iter 280100, loss: 2.134445
----
  song was still and emen we Bric theal ring reone — lithe song was tove”.
Many im in a dueat the chotutei— ligabely bast. Finaly Hendrixt the refar the riths in a spat hound Baker ins roupen-nee 
----
iter 280200, loss: 2.112846
----
 ng ho werk as Lonapping was they they thit the song was show. The hast a ly; the riff. (InFis a vere of Nathing grilad he Brich shts on an Claduters a veasing ist fminative shopt’s drommer eans at L 
----
iter 280300, loss: 2.076458
----
  1969 Montreude apon his stilling w’s lock-boow,

----
 erment and in’s with eririchon and Baken ly in the styl. Ind in the shome pritut we”. Mot version of Nation of “Sunshine of Your Love’s all-night writing rals. (ists thardd this inste as the p 
----
iter 283500, loss: 3.836876
----
  2014; last November a veast thring way live. Foonghale of sst Pein foot 1969 the singin’s an uhise gashinely emerif pind fazzserallirson aw gis Wilson’s toil, ancererser, shige, his be vmt wrudio 
----
iter 283600, loss: 3.782046
----
 os — riguted thuin the riff to rummer Ginger ered.”. Foote, to sho pe. Ther theil supers way to the rinf. Than frement and the producer to ed in the shoped relded came frow. To Broct is and seroly 
----
iter 283700, loss: 3.670969
----
 as born wrmoans a song and swoon Cladudinallistilisenong undrd; on plyrich dp the byron Cream’s drummer Ginger Baker — suggested playing the song the Brume, Claptonceras.
MidwaR gusk aended by a s 
----
iter 283800, loss: 3.555813
----
 out 15 musiciandl sho, tawerg the song was bring a

----
 instere and caled by the US and the chon — some a song, Baker bar, the sunsir, ho werere; nesteri“Hey on therudiore ins red.
Mageaf rubord it in New York, shordy,.g thit on tomrinixt is af ad erow 
----
iter 287000, loss: 2.559493
----
 n Janum Rowa lisply bakenong a ncared through the songon’s Rounduritusted whine tract is wns baintard vest out buested the pood insa next. Spor abo,  pelly,.. They bassinger souldertsistal verter on 
----
iter 287100, loss: 2.617240
----
 endrix — have covered in Navin caste in the prodristiguded Loninglightating a nce to the Brock live shows, from playing that the lisht by balm fe war dith treringer and his bas inher voror”. Finkt 
----
iter 287200, loss: 2.791590
----
 Iuperg ruld-ter version ry the chove”. Footads. Fisomd awads — som tower in an liaktering. A hered peyMidzane a and Iher bans on an Claptoistsus vere wmel guprule of Native Ameror gule shire, Clap 
----
iter 287300, loss: 3.176272
----
  meant to join ther, on on the studio hoppen

----
 ppening a dithed the Jimi he hos cootal salt-ped bruyris shirute, andinglyred relsalman tributhis the sky began to lightruce and stume from, s his staple of Cream’s live host ort sharidicom tied oro 
----
iter 290500, loss: 2.189893
----
 up they may bl shot in a his Hendrix — have riancered pey banddlgs — lnande Amer stidiconogg a version on 1967 as Lubleas ous but in 2014; las Luluteran sowes Wharidudins Haste a nce,. Ebaly in th 
----
iter 290600, loss: 2.143939
----
 ��s a”. a aldyong on this curuiwhon meabe ilisood its rack a Hen the s of wat A Happ, in 1968 an American soul singer by the camerond Brown warert’s dromm a complyex Bricnce and band a from his He 
----
iter 290700, loss: 2.113005
----
  played the endBrifus Lulugersoor the riff a shozs Lulute, andingly the s of laithere this “It’s dy band recore from thos tuims the riff on his Hended a Hanct Han f.g stig, tomers but by the poet  
----
iter 290800, loss: 2.086453
----
 enge. Foocg sallmern Brute-d a Henter ff we 

KeyboardInterrupt: 